### Scraping Data from Amazon using the special code which amazon assigns it to the products called as ASIN number. It can be easily found on the product page in Product details scetion or in the product site url.

#### using Beautifulsoup, Pandas and Requests library from python

In [ ]:
# importing libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [ ]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}
reviewList = []
prodId = 1

In [ ]:
def getSoup(url):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

# this function extracts the review data from a given soup data
def getReviews(soup):
    global prodId
    reviews = soup.find_all('div', {'data-hook':'review'})
    
    try:
        for item in reviews:
            review = {
                'Id': prodId,
                'reviewTitle': item.find('a', {'data-hook':'review-title'}).text.strip(),
                'rating': item.find('i', {'data-hook':'review-star-rating'}).text.replace('out of 5 stars','').strip(),
                'reviewText': item.find('span', {'data-hook':'review-body'}).text.replace('\n',' ').strip()
            }
            prodId += 1
            reviewList.append(review)
    except:
        pass


In [ ]:
def main():
  asin = input('Enter ASIN number of product: ')

  x = 1
  url = f'https://www.amazon.in/product-reviews/{asin}/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber={x}'
  headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}
  page = requests.get(url, headers=headers)
  soup = BeautifulSoup(page.content, 'html.parser')

  # find the element containing the average rating and extract the rating value
  avg_rating_elem = soup.find('span', {'data-hook': 'rating-out-of-text'})
  avg_rating = avg_rating_elem.text.strip()

  # print the average rating
  print(f'Average customer rating: {avg_rating}')
  soup = getSoup(url)
  productName = soup.find('a', {'data-hook':'product-link'}).text.strip()
  print(f'Product: {productName}')

  # getting recent reviews from all the review pages
  for x in range(1,10):
      soup = getSoup(f'https://www.amazon.in/product-reviews/{asin}/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber={x}')
      print(f'Getting review page {x}...')
      getReviews(soup)
      if soup.find('li', {'class':'a-disabled a-last'}):
          break 
      else:
          pass
    

  # converting list into a dataframe    
  df = pd.DataFrame(reviewList)

  # converting dataframe to an excel sheet
  print('Converting to .xlsx ...')
  df.to_excel(asin+'-reviews.xlsx', index=False)
  print('Review extraction finished! extracted to file ' + asin +'-review.xlsx :)')

In [ ]:
if __name__== '__main__':
  main()

Enter ASIN number of product: B01J0XWYKQ
Average customer rating: 4.4 out of 5
Product: Logitech B170 Wireless Mouse, 2.4 GHz with USB Nano Receiver, Optical Tracking, 12-Months Battery Life, Ambidextrous, PC/Mac/Laptop - Black
Getting review page 1...
Getting review page 2...
Getting review page 3...
Getting review page 4...
Getting review page 5...
Getting review page 6...
Getting review page 7...
Getting review page 8...
Getting review page 9...
Converting to .xlsx ...
Review extraction finished! extracted to file B01J0XWYKQ-review.xlsx :)
